In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
import dateutil.easter as easter

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
from accelerate import Accelerator
import torch.optim as optim

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import time

In [7]:
from tqdm.notebook import tqdm

In [8]:
import gc

## Global Variables ###

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
device

device(type='cuda', index=0)

In [11]:
from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [12]:
BASE_DIR = '/sharedHDD/rohit/timeseries_learning/ubiquant/'
DATA_DIR = BASE_DIR+'data/parquet/'
INPUT_DIR = BASE_DIR+'input/'
WEIGHTS_DIR = BASE_DIR + 'weights/'

In [13]:
input_chunk_length = 30
output_chunk_length = 1
embedding_dim = 56 ## Investment id embedding dim

In [14]:
features = [f'f_{i}' for i in range(300)]

## Load data ##

In [15]:
%%time
train_df = pd.read_parquet(DATA_DIR+'train_low_mem.parquet')

CPU times: user 9.04 s, sys: 14 s, total: 23 s
Wall time: 3.77 s


In [16]:
investment_ids = train_df.investment_id.unique().tolist()

In [17]:
max(investment_ids)

3773

In [18]:
with open(BASE_DIR+'input/folds.pickle', 'rb') as f:
    folds = pickle.load(f)

#### Utility functions ######

In [19]:
all_features_columns = ['target'] #+ features

In [20]:
def get_feature_array_train(rows):
    target_val = rows.target.values
    
    inp_vec = rows[all_features_columns].values
    
    future_covariates = rows[['investment_id']+features].values
    
    return inp_vec,target_val,future_covariates

In [21]:
def sliding_windows(X,Y,FCVs, seq_length,prediction_length):
    x = []
    y = []
    future_covariates = []
    for i in range(len(X)-(prediction_length+seq_length)):
        _x = X[i:(i+seq_length),:].reshape(seq_length,-1)
#         _y = Y[(i+seq_length):(i+prediction_length+seq_length),:]
        _y = Y[i+seq_length]
        _future_covariate = FCVs[i+seq_length]
        x.append(_x)
        y.append(_y)
        future_covariates.append(_future_covariate)

    return x,y,future_covariates

In [22]:
def make_sequences(df,to_exclude_ids):
    x = []
    y = []
    future_covariates = []
    investment_ids = df.investment_id.unique()
    for iid in investment_ids:
        if iid not in to_exclude_ids:
            df_tmp = df[df.investment_id == iid].sort_values('time_id')
            tmp_X,tmp_Y,tmp_fcvs = get_feature_array_train(df_tmp)
            _x,_y,_fcv = sliding_windows(tmp_X,tmp_Y,tmp_fcvs,input_chunk_length,output_chunk_length)
            x.extend(_x)
            y.extend(_y)
            future_covariates.extend(_fcv)
            
#             gc.collect()
        # else:
        #     print('Not including iid:', iid)
    
    return np.float32(np.array(x)),np.float32(np.array(y)),np.float32(np.array(future_covariates))

In [23]:
gc.collect()

40

## Modeling ##

In [24]:
config = {
    'seq_length' : input_chunk_length,
    'num_epochs' : 100,
    'lr' : 0.00026,
    'input_size' : 1,
    'hidden_size' : 25,
    'num_layers' : 1,
    'num_classes' :1, ## This is  output dimension
    'train_shuffle': True,
    'val_shuffle': True,
    'batch_size' : 4096*2,
    'best_model_name' : 'lstm3',
    'bidirectional' : False,
    'only_last_hidden': True,
    'early_stopping_patience':10,
}

In [25]:
class TSDataset(Dataset):
    
    def __init__(self,x,y,future_covariates):
        """
        Args:
        """
        self.x=x
        self.y=y
        self.future_covariates = future_covariates

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        
        sample = [torch.Tensor(self.x[idx]),torch.Tensor(self.y[idx]),torch.Tensor(self.future_covariates[idx])]
        return sample

#### Model ####

In [26]:
num_epochs = config['num_epochs']
lr = config['lr']
input_size = config['input_size']
hidden_size = config['hidden_size']
num_layers = config['num_layers']
num_classes = config['num_classes']
seq_length = config['seq_length']
bidirectional = config['bidirectional']
only_last_hidden = config['only_last_hidden']
early_stopping_patience = config['early_stopping_patience']

#### Loss function ######

In [27]:
def pearson_loss(x, y):
    xd = x - x.mean()
    yd = y - y.mean()
    nom = (xd * yd).sum()
    denom = ((xd ** 2).sum() * (yd ** 2).sum()).sqrt()
    return 1 - nom / denom

In [28]:
class LstmTsModel(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers,seq_length):
        super(LstmTsModel, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        cat_input_dim: int = 3774
        
        self.embedding = nn.Embedding(cat_input_dim, embedding_dim)
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,bidirectional=bidirectional)
        
        if bidirectional:
            m=2
        else:
            m=1
        
        if only_last_hidden:
            input_fc_dim = (hidden_size*m)+(len(features)+embedding_dim)
        else:
            input_fc_dim = (self.seq_length*hidden_size*m)+(len(features)+embedding_dim)
        
        print(input_fc_dim)
        
        self.fc = nn.Sequential(nn.Linear(input_fc_dim, 508),
#                                 nn.BatchNorm1d(num_features=508),
                                nn.ReLU(),
                                nn.Dropout(0.42),
                                
                                nn.Linear(508, 405),
#                                 nn.BatchNorm1d(num_features=405),
                                nn.Dropout(0.42),
                                nn.ReLU(),
                                
#                                 nn.Linear(input_fc_dim//16, input_fc_dim//32),
#                                 nn.BatchNorm1d(num_features=input_fc_dim//32),
#                                 # nn.Dropout(0.2),
#                                 nn.ReLU(),
                                
                                
                                nn.Linear(405, self.num_classes)
                                )
    
    def forward(self, x, fcv):
                                # Propagate input through LSTM
        h_out, (h_n, _) = self.lstm(x)
        if only_last_hidden:
            h_out = h_out[:,-1:,:]
        
        h_out = h_out.flatten(start_dim=1)
        
        
        investment_ids = fcv[:,0].type(torch.LongTensor).to(device)
        embeddings = self.embedding(investment_ids)
        
        h_out = torch.cat([embeddings,fcv[:,1:],h_out], dim=1)
        
        out = self.fc(h_out)
        
        return out

In [29]:
def run(model,train_dl,val_dl,fold):
    def evaluate(model,valid_loader):
        model.eval()
        valid_loss = 0
        rec_loss = 0
        with torch.no_grad():
            for i, inputs in enumerate(tqdm(valid_loader)):
                dataX = inputs[0]
                dataY = inputs[1]
                dataFCV = inputs[2]
                outputs = model(dataX,dataFCV)
                loss = criterion(outputs, dataY)
                valid_loss += loss.item()

        valid_loss /= len(valid_loader)
        return valid_loss
    
    def train_and_evaluate_loop(train_loader,model,optimizer,criterion,epoch,lr_scheduler=None,valid_loader=None, best_loss=99999):
        train_loss = 0
        improvement = False
        for i, inputs in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            model.train()
            
            dataX = inputs[0]
            dataY = inputs[1]
            dataFCV = inputs[2]
            outputs = model(dataX,dataFCV)
            loss = criterion(outputs, dataY)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if lr_scheduler:
                lr_scheduler.step()
        
        train_loss /= len(train_loader)
        if valid_loader:
            valid_loss = evaluate(model,valid_loader) 
            print(f"Epoch:{epoch} |Train Loss:{train_loss}|Valid Loss:{valid_loss}")
            if valid_loss <= best_loss:
                print(f"{g_}Loss Decreased from {best_loss} to {valid_loss}{sr_}")

                best_loss = valid_loss
                torch.save(model.state_dict(), WEIGHTS_DIR+str(fold)+'_'+config['best_model_name'])
                improvement = True
        else:
            print(f"Epoch:{epoch} |Train Loss:{train_loss}")
            
                    
        return best_loss,improvement
    
    accelerator = Accelerator()
    print(f"{accelerator.device} is used")

    
    
    optimizer = optim.Adam(model.parameters(),lr=config['lr'],amsgrad=False)
    criterion = pearson_loss
    
    # lr_scheduler = CosineAnnealingWarmupRestarts(optimizer, **config_lr)
    # lr_scheduler =  torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, **config_lr)
    lr_scheduler = None

    model,train_dl,val_dl,optimizer,lr_scheduler,criterion = accelerator.prepare(model,train_dl,val_dl,optimizer,lr_scheduler,criterion)

    best_loss = 9999999
    start_time = time.time()
    current_patience = 0
    for epoch in tqdm(range(config["num_epochs"])):
        print(f"Epoch Started:{epoch}")
        best_loss,improvement = train_and_evaluate_loop(train_dl,model,optimizer,criterion,epoch,lr_scheduler,valid_loader=val_dl, best_loss=best_loss)
        
        if not improvement:
            current_patience+=1
        else:
            current_patience = 0
            
        
        if current_patience == early_stopping_patience:
            print(f'{g_}EARLY STOPPING')
            break
            
            
        
        end_time = time.time()
        print(f"{m_}Time taken by epoch {epoch} is {end_time-start_time:.2f}s{sr_}")
        start_time = end_time
        
    return best_loss, model

In [30]:
gc.collect()

20

In [31]:
# for fold in folds.keys():
for fold in [1,2,3,4]:
    print(f'Starting for fold: {fold}{r_}')
    print(f'Preparing training data for fold: {fold}{m_}')
    train_indxs = folds[fold]['train']
    test_indxs = folds[fold]['test']
    test_f_df = train_df[train_df.index.isin(test_indxs)].reset_index(drop=True)
    train_f_df = train_df[train_df.index.isin(train_indxs)].reset_index(drop=True)
    
    print(train_f_df.shape)
    count_df = train_f_df.groupby('investment_id',as_index=False).agg(c = ('time_id', len))
    to_exclude_ids = count_df[count_df.c<input_chunk_length+1].investment_id.unique().tolist()
    
    X_train, Y_train, FCV_train = make_sequences(train_f_df,to_exclude_ids)
    Y_train = Y_train.reshape(-1,1)
    print(X_train.shape,Y_train.shape,FCV_train.shape)
    
    X_val, Y_val, FCV_val = make_sequences(test_f_df,to_exclude_ids)
    Y_val = Y_val.reshape(-1,1)
    print(X_val.shape,Y_val.shape,FCV_train.shape)
    
    model = LstmTsModel(num_classes, input_size, hidden_size, num_layers,seq_length)
    train_dl = DataLoader(TSDataset(X_train, Y_train, FCV_train), batch_size=config['batch_size'], shuffle=config['train_shuffle'], num_workers=0)
    val_dl = DataLoader(TSDataset(X_val, Y_val, FCV_val), batch_size=config['batch_size'], shuffle=config['train_shuffle'], num_workers=0)
    
    best_loss, model = run(model,train_dl,val_dl,fold)
    
    gc.collect()
    

Starting for fold: 1
Preparing training data for fold: 1
(1053141, 304)
(942773, 30, 1) (942773, 1) (942773, 301)
(412467, 30, 1) (412467, 1) (942773, 301)
381
cuda is used


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch Started:0


  0%|          | 0/116 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

...

  0%|          | 0/51 [00:00<?, ?it/s]

Epoch:16 |Train Loss:0.7239915060686366|Valid Loss:0.8174341075560626
Time taken by epoch 16 is 85.42s
Epoch Started:17


  0%|          | 0/307 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

Epoch:17 |Train Loss:0.7198509599176214|Valid Loss:0.8131377626867855
EARLY STOPPING
